In [1]:
from os import listdir 
import numpy as np
import pandas as pd
import scipy as sp
import regex as re 
from scipy.spatial import Voronoi, voronoi_plot_2d
import seaborn as sb
import nltk
import heapq
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB 

## chargement des données 

In [38]:
def build_df(set='train', lang='en'):
    '''
    Fonction pour lire tous les fichiers dans un dataframe.
    input :
        set : Le set à construire - 'train' ou 'test'.
        lang : La langue du set à construire - 'en' ou 'fr'.
    '''
    res = pd.DataFrame(columns=['content', 'label'])
    
    if lang=='en':
        path = 'data/movie-reviews-en/'+set
    elif lang=='fr':
        path = 'data/movie-reviews-fr/'+set
    pos_path = path+'/pos/'
    neg_path = path+'/neg/'
        
    files = sorted(listdir(pos_path))
    for file in files:
        with open(pos_path+file, 'r',encoding="utf8") as f:
            df = pd.DataFrame({'content':[f.read()], 'label':['pos']})
            res = res.append(df, ignore_index=True)

    files = sorted(listdir(neg_path))
    for file in files:
        with open(neg_path+file, 'r',encoding="utf8") as f:
            df = pd.DataFrame({'content':[f.read()], 'label':['neg']})
            res = res.append(df, ignore_index=True)
    
    return res

In [39]:
a_train = build_df(set='train', lang='en')
a_test = build_df(set='test', lang='en')

In [40]:
a_train

,content,label
0,films adapted from comic books have had plenty...,pos
1,every now and then a movie comes along from a ...,pos
2,you've got mail works alot better than it dese...,pos
3,""" jaws "" is a rare film that grabs your atten...",pos
4,moviemaking is a lot like being the general ma...,pos
...,...,...
1795,2 days in the valley is more or less a pulp fi...,neg
1796,what would inspire someone who cannot write or...,neg
1797,synopsis : a novelist struggling with his late...,neg
1798,"okay , okay . \nmaybe i wasn't in the mood to ...",neg


## 1. Test avec scikitlearn

In [67]:
a_train_pos = a_train[a_train['label']=='pos'] #documents de la classe pos 
a_train_neg = a_train[a_train['label']=='neg'] #documents de la classe neg

In [69]:
def pourcentage(predictions, test):
    a=0
    for i in range(len(predictions)):
        if(test['label'][i]==predictions[i]):
            a+=1 
    b = a/len(predictions)*100.0
    return(b)

In [70]:
vectorizer = CountVectorizer()
 
counts = vectorizer.fit_transform(a_train['content'].values)

In [71]:
classifier = MultinomialNB()
targets = a_train['label'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [72]:
test = vectorizer.transform(a_test['content'])
predictions = classifier.predict(test)

In [73]:
print('Le pourcentage de bonnes prédictions avec la libraire scikitlearn est de : ', pourcentage(predictions,a_test), '%')

Le pourcentage de bonnes prédictions avec la libraire scikitlearn est de :  83.0 %


## 2. Implémentation du code

### A. Le compte des mots 

In [48]:
def traitement_texte(L):
    for i in range(len(L)): #traitement de textes 
        L[i] = L[i].lower() #lettres en micuscules
        L[i] = re.sub(r'\W',' ',L[i]) #suppression de la ponctuation 
        L[i] = re.sub(r'\s+',' ',L[i]) #suppresion de espaces blancs
    return L

In [49]:
def freqMots(df):
    l = []
    dic = {}
    for e in df.itertuples():
        l.append(e[1])
    L = traitement_texte(l)
    for phr in L: #création du dictionnaire
        mots = nltk.word_tokenize(phr)
        for mot in mots:
            if mot not in dic.keys():
                dic[mot] = 1
            else:
                dic[mot] += 1
    return dic

In [89]:
dic = freqMots(a_train) #dictionnaire qu'on pourra utiliser comme vocabulaire

In [51]:
dic_pos = freqMots(a_train_pos) #mots de la classe pos 
dic_neg = freqMots(a_train_neg) #mots de la classe neg

In [52]:
nbr_mots_pos=0 #nombre de mots total dans la classe pos
for valeur in dic_pos.values():
    nbr_mots_pos+=valeur
    
nbr_mots_neg=0 #nombre de mots total dans la classe neg
for valeur in dic_neg.values():
    nbr_mots_neg+=valeur

### B. Calcul des probabilités

In [77]:
def conditionnal_prob(dic):
    """
        fonction qui va calculer les probabilités conditionnelles de chaque mot selon la classe dont elle appartient
    """
    dpos_prob = {} #dictionnaire qui contiendra les probabilités du vocabulaire pour la classe pos
    dneg_prob = {} #dictionnaire qui contiendra les probabilités du vocabulaire pour la classe neg
    a = 0
    for cle, valeur in dic.items():
        if cle in dic_pos:
            a = (dic_pos[cle]+1.0)/(nbr_mots_pos+1) #utilisation de la formule 
            dpos_prob[cle]=a
        else: #si le mot n'appartient pas à la classe 
            a = 1.0/(nbr_mots_pos+1)
            dpos_prob[cle]=a
        if cle in dic_neg:
            a = (dic_neg[cle]+1.0)/(nbr_mots_neg+1)
            dneg_prob[cle]=a
        else: 
            a = 1.0/(nbr_mots_neg+1)
            dneg_prob[cle]=a
    return dpos_prob, dneg_prob 

In [78]:
dpos_prob, dneg_prob = conditionnal_prob(dic)

### C. Choix de la classe

In [79]:
def classe(df): 
    """
        fonction qui va déterminer selon la base d'apprentissage si un document de la base de test
        est de la classe pos ou neg
    """
    L=[]
    Lp = []
    Ln = []
    l = []
    for e in df.itertuples():
        L.append(e[1])
    for i in range(len(df)): #traitement de textes 
        L[i] = L[i].lower() #lettres en micuscules
        L[i] = re.sub(r'\W',' ',L[i]) #suppression de la ponctuation 
        L[i] = re.sub(r'\s+',' ',L[i]) #suppresion de espaces blancs
        #compte des mots 
        mots = L[i].split()
        compte = {}.fromkeys(set(mots),0)
        for valeur in mots:
            compte[valeur] += 1
        #Propriétés d'appartenir à la classe pos ou neg
        pp = np.log(0.5) #0.5 est une prior probability pour la classe pos 
        pn = np.log(0.5) #0.5 est une prior probability pour la classe neg
        for cle in compte.keys():
            if cle in dic: 
                pp+=np.log(dpos_prob[cle]**(compte[cle])) 
                #en puissance compte[cle] correspond à la fréquence du mot dans le document
            if cle in dic: 
                pn+=np.log(dneg_prob[cle]**(compte[cle]))
        Lp.append(pp)
        Ln.append(pn)
    for i in range(len(Lp)): 
        if(Lp[i]>Ln[i]):
            l.append('pos')
        else:
            l.append('neg')
    return l

In [80]:
pred = classe(a_test)

In [81]:
print('Le pourcentage de bonnes prédictions est de : ', pourcentage(pred,a_test), '%')

Le pourcentage de bonnes prédictions est de :  81.5 %


In [82]:
pred_pos = classe(a_test[a_test['label']=='pos'])
print('Le pourcentage de bonnes prédictions pour la classe pos est de : ', pourcentage(pred_pos,a_test[a_test['label']=='pos']), '%')

Le pourcentage de bonnes prédictions pour la classe pos est de :  73.0 %


In [83]:
pred_neg = classe(a_test[a_test['label']=='neg'])
c = a_test[a_test['label']=='neg'].reset_index(drop = True)
print('Le pourcentage de bonnes prédictions pour la classe neg est de : ', pourcentage(pred_neg, c), '%')

Le pourcentage de bonnes prédictions pour la classe neg est de :  90.0 %


#### Test avec les données françaises 

In [84]:
b_train = build_df(set='train', lang='fr')
b_test = build_df(set='test', lang='fr')

In [85]:
dic = freqMots(b_train) #dictionnaire qu'on pourra utiliser comme vocabulaire

b_train_pos = b_train[b_train['label']=='pos'] #documents de la classe pos 
b_train_neg = b_train[b_train['label']=='neg'] #documents de la classe neg 

dic_pos = freqMots(b_train_pos) #mots de la classe pos 
dic_neg = freqMots(b_train_neg) #mots de la classe neg

nbr_mots_pos=0 #nombre de mots total dans la classe pos
for valeur in dic_pos.values():
    nbr_mots_pos+=valeur
    
nbr_mots_neg=0 #nombre de mots total dans la classe neg
for valeur in dic_neg.values():
    nbr_mots_neg+=valeur

In [86]:
dpos_prob, dneg_prob = conditionnal_prob(dic)

In [87]:
pred = classe(b_test)

In [88]:
print('Le pourcentage de bonnes prédictions est de : ', pourcentage(pred,b_test), '%')

Le pourcentage de bonnes prédictions est de :  71.5 %
